In [1]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.1/230.1 MB 256.8 kB/s eta 0:00:0000:0100:32
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 455.1 kB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 320.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 455.0 kB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 456.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 463.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 550.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 403.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 411.1 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.2/397.2 kB 499.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 388.6 kB/s eta 0:00:0000:0100:01
  Inst

  Created wheel for jax: filename=jax-0.4.8-py3-none-any.whl size=1439678 sha256=d56436559f8d87de61953209690a494f4e2f525ff8ff500f51f5dc01e0a25cde
  Stored in directory: /Users/xaviersantos/Library/Caches/pip/wheels/05/94/dc/81042da9bced43ff430bc02043d213d9e4b210b584c39e31c1
Successfully built jax
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
      Successfully uninstalled protobuf-3.20.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
tensorboardx 2.5.1 requires protobuf<=3.20.1,>=3.8.0, but you have protobuf 4.22.1 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, 

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, TimeDistributed, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

# Carregue seus dados
data = pd.read_csv('../data/test_gender_fusion.csv')

# Pré-processamento de texto (tokenização e padding)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Description Tokenized'])
sequences = tokenizer.texts_to_sequences(data['Description Tokenized'])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

max_length = max([len(s) for s in sequences])
X_padded = pad_sequences(sequences, maxlen=max_length, padding='post')

# Labels (gêneros)
Y = data.iloc[:, 7:18].values 

# Divida os dados em conjuntos de treinamento e teste
from sklearn.model_selection import train_test_split
X_train_padded, X_test_padded, Y_train, Y_test = train_test_split(X_padded, Y, test_size=0.2, random_state=42)

# Crie o modelo LSTM unidirecional
embedding_dim = 100

num_genres = 11

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(LSTM(128, return_sequences=True, dropout=0.25, recurrent_dropout=0.25))
model.add(TimeDistributed(Dense(128, activation='relu')))
model.add(Dropout(0.5))
model.add(LSTM(128, dropout=0.25, recurrent_dropout=0.25))
model.add(Dense(num_genres, activation='sigmoid'))

# Compile e treine o modelo
optimizer = RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

num_epochs = 15
history = model.fit(X_train_padded, Y_train, epochs=num_epochs, validation_data=(X_test_padded, Y_test), verbose=2)

# Avalie o modelo
scores = model.evaluate(X_test_padded, Y_test, verbose=2)

2023-04-10 18:27:52.234282: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/15
201/201 - 64s - loss: 0.4590 - accuracy: 0.3050 - val_loss: 0.4416 - val_accuracy: 0.3059 - 64s/epoch - 317ms/step
Epoch 2/15
201/201 - 58s - loss: 0.4524 - accuracy: 0.3159 - val_loss: 0.4441 - val_accuracy: 0.3059 - 58s/epoch - 287ms/step
Epoch 3/15
201/201 - 59s - loss: 0.4519 - accuracy: 0.3159 - val_loss: 0.4424 - val_accuracy: 0.3059 - 59s/epoch - 295ms/step
Epoch 4/15
201/201 - 54s - loss: 0.4484 - accuracy: 0.3137 - val_loss: 0.4289 - val_accuracy: 0.3059 - 54s/epoch - 270ms/step
Epoch 5/15
201/201 - 52s - loss: 0.4342 - accuracy: 0.3061 - val_loss: 0.4180 - val_accuracy: 0.2636 - 52s/epoch - 258ms/step
Epoch 6/15
201/201 - 57s - loss: 0.4148 - accuracy: 0.2890 - val_loss: 0.4177 - val_accuracy: 0.3040 - 57s/epoch - 286ms/step
Epoch 7/15
201/201 - 55s - loss: 0.4022 - accuracy: 0.2915 - val_loss: 0.4318 - val_accuracy: 0.2380 - 55s/epoch - 274ms/step
Epoch 8/15
201/201 - 59s - loss: 0.3924 - accuracy: 0.2958 - val_loss: 0.4089 - val_accuracy: 0.2822 - 59s/epoch - 296